# Detect sentiment in customer calls using Amazon Comprehend

Now we will detect the customer sentiment in the call conversations using Amazon Comprehend. 

### Import libraries and initialize variables

In [ ]:
import boto3
import pandas as pd

inprefix = 'comprehend/input'
outprefix = 'quicksight/temp/insights'
# Amazon Comprehend client
comprehend = boto3.client('comprehend')
# Amazon S3 clients
s3 = boto3.client('s3')
s3_resource = boto3.resource('s3')

bucket = '<your-bucket-name>' # Enter your bucket name here

try:
    s3.head_bucket(Bucket=bucket)
except:
    print("The S3 bucket name {} you entered seems to be incorrect, please try again".format(bucket))

### Detect sentiment of transcripts
Write about Comprehend detect sentiments here...

In [ ]:
# Prepare to page through our transcripts in S3
paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucket, Prefix=inprefix)
job_name_list = []
t_prefix = 'quicksight/data/sentiment'

# We will define a DataFrame to store the results of the sentiment analysis
cols = ['transcript_name', 'sentiment']
df_sent = pd.DataFrame(columns=cols)

# Now lets page through the transcripts
for page in pages:
    for obj in page['Contents']:
        # get the transcript file name
        transcript_file_name = obj['Key'].split('/')[2]
        # now lets get the transcript file contents
        temp = s3_resource.Object(bucket, obj['Key'])
        transcript_contents = temp.get()['Body'].read().decode('utf-8')
        # Call Comprehend to detect sentiment
        response = comprehend.detect_sentiment(Text=transcript_contents, LanguageCode='en')
        # Update the results DataFrame with the cta predicted label
        # Create a CSV file with cta label from this DataFrame
        df_sent.loc[len(df_sent.index)] = [transcript_file_name.strip('en-').strip('.txt'),response['Sentiment']]
        
df_sent.to_csv('s3://' + bucket + '/' + t_prefix + '/' + 'sentiment.csv', index=False)
df_sent

## End of notebook. Please go back to the workshop instructions to review the next steps.